# Подбор гипперпараметров нейросети при помощи Keras Tuner Bayesian Optimization

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD, Adam, Adadelta, RMSprop
from keras_tuner import RandomSearch, BayesianOptimization, Objective

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from python_files.data import data
data_ = data.copy()

In [3]:
from python_files.matrix.mat_split_data import X_train_mat, X_test_mat, y_train_mat, y_test_mat
from python_files.matrix.mat_col_list import mat_x_col_list, mat_col_list_norm, mat_col_list_std

### Preprocessing

In [4]:
# нормализуем и стандартизируем X_train, X_test
X_train_mat_prep = PowerTransformer().fit_transform(X_train_mat) 
X_test_mat_prep = PowerTransformer().fit_transform(X_test_mat) 

In [5]:
# нормализуем и стандартизируем y_train, y_test
y_train_mat_df = pd.DataFrame(y_train_mat, columns = [data_.columns[0]])
y_test_mat_df = pd.DataFrame(y_test_mat, columns = [data_.columns[0]])

pt_y_train = PowerTransformer()
pt_y_test = PowerTransformer()

y_train_mat_prep = pt_y_train.fit_transform(y_train_mat_df) 
y_test_mat_prep = pt_y_test.fit_transform(y_test_mat_df) 

### Model - MLP

In [14]:
# определяем пользовательскую функцию - нейросеть  
def MLP_model(hp):
    model = Sequential()
    model.add(Dense(units = hp.Float('units_input_1', min_value = 8, max_value = 48, step = 4),
                            activation = hp.Choice('activation_1', values = ['relu','tanh','sigmoid','elu']),
                            input_shape = (12,), 
                            bias_initializer = hp.Choice('bs_init_1', values = ['zeros', 'he_normal', 'glorot_normal']),
                            kernel_initializer = hp.Choice('kn_init_1', values = ['he_normal', 'glorot_normal'])))
    
    model.add(Dropout(rate = hp.Float('dropout_1', min_value = 0.3, max_value = 0.6, step = 0.1)))
    
    model.add(Dense(units = hp.Float('units_input_2', min_value = 8, max_value = 48, step = 4),
                            activation = hp.Choice('activation_2', values = ['relu','tanh','sigmoid','elu']),
                            bias_initializer = hp.Choice('bs_init_2', values = ['zeros', 'he_normal', 'glorot_normal']),
                            kernel_initializer = hp.Choice('kn_init_2', values = ['he_normal', 'glorot_normal'])))   
    
    model.add(Dropout(rate = hp.Float('dropout_2', min_value = 0.0, max_value = 0.6, step = 0.2)))
    
    model.add(Dense(1, activation = 'linear', 
                    bias_initializer = hp.Choice('bs_init_ex', values = ['zeros', 'he_normal']),
                    kernel_initializer = 'he_normal'))
    
    sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
              
    model.compile(loss = 'mean_absolute_error', optimizer = hp.Choice('optimizer', values = ['sgd','Adam','Adadelta','RMSprop']),
                  metrics=['mean_absolute_error'])
    return model

### Keras Tuner

In [15]:
# определяем кeras_tuner
tuner = BayesianOptimization(
                     MLP_model,
                     objective = Objective("val_mean_absolute_error", direction="min"),
                     max_trials = 300,
                     directory = '/keras_tuner/untitled_project',
                     overwrite = True
                    )

In [16]:
# пространство подбираемых гипперпараметров
tuner.search_space_summary()

Search space summary
Default search space size: 12
units_input_1 (Float)
{'default': 8.0, 'conditions': [], 'min_value': 8.0, 'max_value': 48.0, 'step': 4, 'sampling': 'linear'}
activation_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid', 'elu'], 'ordered': False}
bs_init_1 (Choice)
{'default': 'zeros', 'conditions': [], 'values': ['zeros', 'he_normal', 'glorot_normal'], 'ordered': False}
kn_init_1 (Choice)
{'default': 'he_normal', 'conditions': [], 'values': ['he_normal', 'glorot_normal'], 'ordered': False}
dropout_1 (Float)
{'default': 0.3, 'conditions': [], 'min_value': 0.3, 'max_value': 0.6, 'step': 0.1, 'sampling': 'linear'}
units_input_2 (Float)
{'default': 8.0, 'conditions': [], 'min_value': 8.0, 'max_value': 48.0, 'step': 4, 'sampling': 'linear'}
activation_2 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid', 'elu'], 'ordered': False}
bs_init_2 (Choice)
{'default': 'zeros', 'conditions': [], 'values': ['zero

In [17]:
# подбираем гипперпараметры
tuner.search(X_train_mat_prep, y_train_mat_prep,
             epochs = 200,
             validation_split = 0.3,
             verbose = 1,
           # directory = '/keras_tuner/untitled_project'             
             )

Trial 300 Complete [00h 00m 27s]
val_mean_absolute_error: 0.7892499566078186

Best val_mean_absolute_error So Far: 0.7728890180587769
Total elapsed time: 03h 41m 15s
INFO:tensorflow:Oracle triggered exit


### Результаты работы Keras Tuner

In [18]:
# отчет по результатам каждой попытки keras_tuner
tuner.results_summary()

Results summary
Results in /keras_tuner/untitled_project\untitled_project
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 047 summary
Hyperparameters:
units_input_1: 8.0
activation_1: elu
bs_init_1: he_normal
kn_init_1: he_normal
dropout_1: 0.4
units_input_2: 36.0
activation_2: elu
bs_init_2: zeros
kn_init_2: he_normal
dropout_2: 0.2
bs_init_ex: zeros
optimizer: sgd
Score: 0.7728890180587769

Trial 231 summary
Hyperparameters:
units_input_1: 8.0
activation_1: elu
bs_init_1: glorot_normal
kn_init_1: he_normal
dropout_1: 0.3
units_input_2: 8.0
activation_2: relu
bs_init_2: glorot_normal
kn_init_2: he_normal
dropout_2: 0.4
bs_init_ex: he_normal
optimizer: sgd
Score: 0.7772849202156067

Trial 246 summary
Hyperparameters:
units_input_1: 8.0
activation_1: elu
bs_init_1: glorot_normal
kn_init_1: he_normal
dropout_1: 0.5
units_input_2: 20.0
activation_2: elu
bs_init_2: zeros
kn_init_2: glorot_normal
dropout_2: 0.0
bs_init_ex: zeros
optimizer: sgd
Score:

In [19]:
best_hp = tuner.get_best_hyperparameters()[0]

In [20]:
# подобранные гипперпараметры
best_hp.values

{'units_input_1': 8.0,
 'activation_1': 'elu',
 'bs_init_1': 'he_normal',
 'kn_init_1': 'he_normal',
 'dropout_1': 0.4,
 'units_input_2': 36.0,
 'activation_2': 'elu',
 'bs_init_2': 'zeros',
 'kn_init_2': 'he_normal',
 'dropout_2': 0.2,
 'bs_init_ex': 'zeros',
 'optimizer': 'sgd'}

In [21]:
best_model = tuner.get_best_models()[0]

In [22]:
# найденная лучшая архитектура 
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 104       
                                                                 
 dropout (Dropout)           (None, 8)                 0         
                                                                 
 dense_1 (Dense)             (None, 36)                324       
                                                                 
 dropout_1 (Dropout)         (None, 36)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 37        
                                                                 
Total params: 465
Trainable params: 465
Non-trainable params: 0
_________________________________________________________________


### Prediction

In [23]:
preds = best_model.predict(X_test_mat_prep)

10/10 [==============================] - 1s 5ms/step


In [30]:
preds_train = best_model.predict(X_train_mat_prep)

23/23 [==============================] - 0s 2ms/step


### Evaluation

In [31]:
r2_score(y_train_mat_prep, preds_train), r2_score(y_test_mat_prep, preds)

(0.007363817696831587, -0.020332841683557756)

In [25]:
preds_inv = pt_y_test.inverse_transform(preds)

C:\anaconda3\envs\VKR\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but PowerTransformer was fitted with feature names
  warnings.warn(


In [26]:
mean_absolute_error(y_test_mat, preds_inv)

0.6909855685440409

In [27]:
mean_squared_error(y_test_mat, preds_inv)

0.7700226500805638

In [28]:
y_test_mat.max() - y_test_mat.min() 

5.202338993519127

In [29]:
mean_absolute_error(y_test_mat, preds_inv) * 100 / (y_test_mat.max() - y_test_mat.min())

13.28220958697317